******Table of contents******

This notebook is split into two parts: Part 1 and Part 2. You can run the part 1 and part 2 separetely from each other's. This report aims to give some insights of the data from "health_study", which contains the variables age, sex, smoker(No/Yes), disease(0/1), cholesterol, systolic bloodpressure, height, weight, and 800 datapoints (anonymous individuals). 

**Part 1:**
1. Calculate the mean, median, minimum (min), and maximum (max) for the following variables: age, weight, height, systolic_bp, and cholesterol.
2. Create at least 3 different graphs (e.g., a histogram of blood pressure, a box plot of weight per gender, a bar chart of the proportion of smokers).
3. 
- Calculate the proportion of individuals in the dataset who have the disease.
- Use NumPy to simulate 1000 random individuals with the same probability of having the disease.
-  Compare the simulated proportion with the actual proportion found in the dataset.
4. Calculate the confidence interval using two different methods (e.g., normal approximation and bootstrap) and compare the results. (Advanced Requirement - VG)
5. Test the hypothesis: "Smokers have a higher mean blood pressure than non-smokers."
6. Conduct a simulation to examine the reliability of your hypothesis test (e.g., how often the test finds a difference when one truly exists $\rightarrow$ statistical power). (Advanced Requirement - VG)

**Part 2: Advanced Analysis and Pipeline Development**

Code Structure and Modularity
1. Structure the Code: Organize the overall code base logically.
2. Modularization: Move relevant code segments from Part 1 into reusable functions and modules.
3. Object-Oriented Programming (OOP): 
- Create at least one class (e.g., HealthAnalyzer) capable of performing a specific part of the analysis (e.g., calculating summary statistics or plotting graphs).
- Build a more developed class that can handle multiple different analyses or visualizations (Advanced Requirement - VG).
4. Applied Linear Algebra: Use NumPy or scikit-learn for an analysis that relies on matrices/vectors. Utilize a more advanced method (e.g., multiple regression) or include an additional visualization that clearly explains the results (Advanced Requirement - VG).
5. Deeper Insight: Add at least one new analysis or graph that provides a deeper understanding (e.g., the relationship between blood pressure and age, or disease prevalence per gender).
6. Write docstrings for your functions and classes.
7. Use Markdown blocks to explain what you are doing and why.
8. Justify your choice of methods within the Markdown blocks and refer to documentation or other sources (Advanced Requirement - VG).


**Part 1**

**Import and clean data**

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats
from scipy import linalg
import math
from src.moduls import * 
from src.io_utils import *
from src.viz import *
import statsmodels.api as sm

np.random.seed(42)

df = clean_data(read_data("dataset/health_study_dataset.csv"))
df.sample(10)
df.info()

**1. Max, min, mean, median of:**
- Age
- Weight
- Height
- Systolic_bp
- Cholesterol


In [ ]:
showing_standard_info(df)

**2. a) Boxplot of weight grouped by sexes**

**Conclusion: Men are heavier than women when comparing:**
- Mean
- Q1 to Q3, where men's interval is higher than women's
- The normalised interval, where the entire interval (excluding outliers) are higher than women
- Funny sidenote, even when looking at the outliers, the low-weight outliers-men are within the women's normal range, and vice versa for the heavier outliers of women, within the normal range of men.

**Motivation of using boxplot for this scenario:** My main motivation is best described from this quoto: *"Boxplots are built to provide high-level information at a glance, offering general information about a group of data’s symmetry, skew, variance, and outliers. It is easy to see where the main bulk of the data is, and make that comparison between different groups."*

They do however have some limitations, but for this purpose in this specific boxplot, we won't get any of these.

- Källa: https://www.atlassian.com/data/charts/box-plot-complete-guide



In [ ]:
plot_gender_weight_difference(df)

**2. b) Comparing the blood pressure between disease and non-disease people, grouped by sex and smokers**

**Conclusion by only comparing the bars**
- Sick people have, as a group, a higher average bloodpressure than non-sick people in all groups except males who smokes

**Motivation of using barplot:** My choice of barplot is best described with this quote: *"Why Use Bar Plots?
Bar plots are significant because they provide a clear and intuitive way to visualize categorical data. They allow viewers to quickly grasp differences in size or quantity among categories, making them ideal for presenting survey results, sales data, or any discrete variable comparisons."*

And in this case we are comparing different groups (sex and smokers) with the numerical value of systolic bp.

https://www.geeksforgeeks.org/pandas/bar-plot-in-matplotlib/


In [ ]:
tuple_of_healthy_vs_diseased = healthy_vs_diseased_info(df)
plot_disease_vs_healthy(tuple_of_healthy_vs_diseased[1], tuple_of_healthy_vs_diseased[0])

**2. c) Comparing age with cholesterol between disease and non-disease people**

**Conclusion**
- Too few datapoints from the disease-group to actually tell anything but, but looking at age and cholesterol in it's total it seems like cholesterol raises with age.

**Motivation of using scatter-plot:** Scatter plot is generally the best choice when we have two numerical variables that we want to check against each other's to see if there could be any correlation ( https://online.visual-paradigm.com/knowledge/data-visualization/what-is-scatter-diagram/ ). However, to make things even clearer we can also use linear regression, to make a linear function within our graph that tries to be as close to as many datapoints are possible ( https://www.britannica.com/topic/linear-regression ) which can then be used to make predictions. However, it must be noted that linear regression does not check for any statistical significance, and this is a great example of that, that despite us having only 47 datapoints, it draws the linear function anyway. 

In [ ]:
tuple_of_healthy_vs_diseased_mask = healthy_vs_diseased(df)
plot_comparing_age_with_cholesterol(tuple_of_healthy_vs_diseased_mask[0], tuple_of_healthy_vs_diseased_mask[1])

**3. Disease-frequency vs creating a random dataset with the actual disease frequency**

- Blue: Actual disease-frequency
- Yellow: Randomised dataset with the actual frequency in mind
- Red: Difference between the actual and the random frequency

**Motivation of using bar-plot:** Since comparing two categories, one actual and one fictional frequency, we are comparing two groups numerical difference, and that is best plotted with a barplot. See previous motivation from 2 b).

In [ ]:
df_frequency = actual_frequency_vs_random_generated_frequency(df)
plot_actual_frequency_vs_random_generated_frequency(df_frequency)
df_frequency

**4. CI normal approximation and bootstrap of systolic bp**

**Conclusion**

Both bootstrap and normal approximation shows a very similar result, indicating that normal approximation is in this scenario a robust way of calculating a confidence interval. If the results from the two methods were to differ significantly, it would indicate that the Central Limit Theorem (CLT) is not sufficiently satisfied. Either because our n is too small for the sampling distribution of the mean to be approximated as normal, or because the underlying data distribution is highly non-normal (skewed, heavy-tailed), which the bootstrap method is better equipped to handle.

https://www.youtube.com/watch?v=xjYEYBvPaSc&t=2572s


In [ ]:
ci_analysis = CI_and_bootstrap(df["systolic_bp"], confidence=0.95)

results = ci_analysis.compare_methods(B=10000)

results

plot_ci_comparison(results)

**Bonus hypothesis-test**: Bloodpressure in healthy vs disease group differs

- *H0: Bloodpressure in healthy and disease-group is the same*
- *H1: Bloodpressure in healthy and disease-group differs*

**Conclusion**

The test didn't find a p-value < 0.05, but rather a p-value > 0.3 which is way too high for a significant result. Therefore, we cannot reject the null hypothesis.

**Motivation of why using a two tail t-test:** We want to find out if there are any markers that could give an indication of a person being sick, and we don't know how the disease affects systolic bp, so therefore we want to first figure out if there is a difference between the disease group and the non disease group at all, and so we use a two tail test t-test to see if the mean of the disease systolic bp is actually, in a statistical significant way, different from that of the healthy group. 

https://statisticsbyjim.com/hypothesis-testing/one-tailed-two-tailed-hypothesis-tests/ <-- Theory
https://www.youtube.com/watch?v=jfBuCNus-HY <-- Code

In [ ]:
disease, healthy = healthy_vs_diseased(df)

stats.ttest_ind(a=disease["systolic_bp"],
                b=healthy["systolic_bp"],
                equal_var=False)

**Bonus power-test: Checking for how often our t-test will actually find a difference if there is one (type 2 error)**

- Power: 0.177 with difference between sick and healthy set to: 2mmHg
- Power: 0.705 with difference between sick and healthy set to: 5mmHg
- Power: 0.849 with difference between sick and healthy set to: 6mmHg
- Difference set to greater than 6 mmHg generates a power well above 0.85, which is considered a valid number.

**Conclusion:** Our previous t-test will not find small differences between the groups (differences between the group of < 6 mmHg), since our datapoints are too few. However, greater differences between the groups will still be found (> 6 mmHg). If we want our t-test to be more refined for finding even small differences, we need to increase our n greatly.

**Motivation of using power-test:** Since we want to find out what kind of differences our previous t-test will actually be able to find, considering our n, we use a power-test so see how many times our t-test finds significant differences, considering the scenario that there actually is a real difference. This is usually a method you want to use before collecting your sample, since it gives an indication of how large our n needs to be. But since we already have our sample, and our t-test couldn't find a significant difference, we want to see for ourselves how refined our t-test actually was.

https://www.youtube.com/watch?v=fmEk4L9IlA8&t=1s


In [ ]:
# print(disease.describe())     #<-- To check if the correct n and std have been plugged in
# print(healthy.describe())     #<-- To check if the correct n and std have been plugged in

print(checking_power_of_that_t_test(47,753,13.172255, 12.771772, 2, 0.05, 1000, "two-sided"))
print(checking_power_of_that_t_test(47,753,13.172255, 12.771772, 5, 0.05, 1000, "two-sided"))
print(checking_power_of_that_t_test(47,753,13.172255, 12.771772, 6, 0.05, 1000, "two-sided"))
print(checking_power_of_that_t_test(47,753,13.172255, 12.771772, 8, 0.05, 1000, "two-sided"))
print(checking_power_of_that_t_test(47,753,13.172255, 12.771772, 10, 0.05, 1000, "two-sided"))


**5. Trying hypothesis smokers have higher systolic bp than non smokers**

- H0: Smokers does not have higher systolic bp than non smokers
- H1: Smokers do have higher systolic bp than non smokers

**Conclusion:** p-value = 0.326, which means that we cannot reject our null hypothesis, and cannot conclude that smokers do have higher systolic bp than non smokers. 

**Motivation:** Here, we only want to check if smokers have a higher systolic bp than non smokers, and we therefore only use a one tail t-test, specifically checking the right side of our normal approximation, while also assuming that the two groups variance can differ from one another (welch test). 
Links: https://statisticsbyjim.com/hypothesis-testing/one-tailed-two-tailed-hypothesis-tests/ <-- Theory
https://www.youtube.com/watch?v=jfBuCNus-HY <-- Code

In [ ]:
smokers, non_smokers = looking_for_them_smokers(df)


result = stats.ttest_ind(a=smokers["systolic_bp"],
                        b=non_smokers["systolic_bp"],
                        equal_var=False,
                        alternative="greater")

print(result)

**6. Power-testing the previous t-test**

**Conclusion:** Here, in comparison to our previous power test, we can see that we have a sufficient power of finding a the difference that smokers have a higher systolic bp of greater than 3 mmHg. If the difference is less than 3 mmHg, our test wont have sufficient power, and we might miss the difference. 

**Motivation of using a power-test:** As previously stated, since we couldn't find a significant greater mean of systolic bp when comparing smokers to non smokers, and we already have our sample size, it is logical to see how refined our t-test actually was in finding even small differences. Therefore, we do this power-test.

In checking power of that t-test i use a for loop which uses the basics of a t-test, creating some random datasets, doing t-test and viewing if the result came back as significant. However, there is a better method for this i found at https://docs.scipy.org/doc/scipy//reference/generated/scipy.stats.power.html . This does it faster since it doesn't need to loop 1000 times, but since i'm in the beginnig of my statistical journey, my method is easier to follow for a novice. 





In [ ]:
# print(smokers.describe())         #<-- To check if the correct n and std have been plugged in
# print(non_smokers.describe())     #<-- To check if the correct n and std have been plugged in

checking_power_of_that_t_test(213, 587, 13.2678 ,12.626038, 2, 0.05, 1000, "greater")
checking_power_of_that_t_test(213, 587, 13.2678 ,12.626038, 3, 0.05, 1000, "greater")

**Bonus-plot** 

**Notes:** Since having our print in our "checking_power_of_that_t_test" we get some ugly prints when running our plot, since our plot uses the function repeatedly.

**Conclusion:** Here we actually calculate the difference that our t-test will find with a power of 80%, so we find that we will find a difference of 2,5 mmHg.

**Source:** The one and only Joakim -> https://www.youtube.com/watch?v=tAW2Komgulk&t=4164s

In [ ]:
plotting_power_of_that_t_test(213, 587)

**Part 2**

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression 
import matplotlib.pyplot as plt
from scipy import stats
from scipy import linalg
import math
import statsmodels.api as sm

from src.regression_analysis import *
from src.moduls import * 
from src.io_utils import *
from src.viz import *
from src.viz_part2 import *
from src.PCA_and_PCR import *

In [2]:
df = clean_data(read_data("dataset/health_study_dataset.csv"))

NameError: name 'clean_data' is not defined

**Linear regression**

To see how our numerical variables height, weight, cholesterol and age correlate with systolic bp, we do a linear regression where we are interested in our slope (our linear coefficient which tells us how much our independent variable affects our dependent variable) and our r square value (which tells us how much of the variance in our dependent variable is explained by our independent variable).

Below we see the results of our four linear regressions:
1. Our r squared number is way too low, which can also be seen in the plot (that our dots are quite far away from our linear function). This indicates that height might not have too much to do with systolic bp according to our data.
2. Same problem as previous.
3. Now we have our r-squared value at 0.136 which tells us that cholesterol could actually have something to do with systolic bp, but our r square is still too low.
4. Age and systolic bp get's an r-square value of 0.369, which tells us that age and systolic bp could in fact be correlated. And our slope is 0.54, meaning that for every year you age, your systolic bp generally goes up with 0.54 mmHg.

Our next step is to look at our residual diagnostics, to check if our linear regression is satisfying the neccessary conditions of a linear regression.

**Residual Diagnostics**
- Based on the four plots provided for the regression model, the residual diagnostics indicate that the model meets our conditions required for our linear regression.

1. Residuals vs Fitted Plot (Top Left)
The residuals are randomly scattered around the horizontal line at zero.
*Conclusion: This suggests that the linearity assumption is met, and there is no weird patterns visible.*

2. Normal Q-Q Plot (Top Right)
Most data points closely follow the straight red line.
*Conclusion: This confirms that the residuals are approximately normally distributed, which is a crucial assumption for performing hypothesis tests and calculating confidence intervals.*

3. Scale-Location Plot (Bottom Left)
The points are spread evenly across the range of fitted values, showing no weird pattern.
*Conclusion: This indicates homoscedasticity (that we have constant variance).*

4. Histogram of Residuals (Bottom Right)
The distribution is bell-shaped and symmetric around zero.
*Conclusion: This confirms the finding from the Q-Q plot that the residuals are normally distributed.*

**Source:** 
- https://www.youtube.com/watch?v=iMdtTCX2Q70
- https://www.youtube.com/watch?v=sDrAoR17pNM
- https://youtu.be/ZsJ-DbKpD3s?t=2660
- https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLS.html 
- https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.RegressionResults.html <-- För att se hur residuals sparas (resid)
- https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.probplot.html
- https://www.statology.org/standardized-residuals-python/
- Claude and Gemini (helping with the residual diagnostics of Q-Q and scale-location)


In [ ]:


use_reg = RegressionAnalysis(df)

slope, intercept, r2, model = use_reg.linear_regression("height", "systolic_bp")
plotting_that_linear_regression(df, "height", "systolic_bp", slope, intercept)

slope, intercept, r2, model = use_reg.linear_regression("weight", "systolic_bp")
plotting_that_linear_regression(df, "weight", "systolic_bp", slope, intercept)

slope, intercept, r2, model = use_reg.linear_regression("cholesterol", "systolic_bp")
plotting_that_linear_regression(df, "cholesterol", "systolic_bp", slope, intercept)

slope, intercept, r2, model = use_reg.linear_regression("age", "systolic_bp")
plotting_that_linear_regression(df, "age", "systolic_bp", slope, intercept)
plot_residual_diagnostics(model)
print(model.summary())

**Multiple linear regression**

To see how our different variables correlate with systolic bp, we do a multiple regression, which isolates one variable at a time and check how that variables unique influence correlates with systolic bp, where we both get a coefficient (how much the isolated variable affects systolic bp) and a p value, which tells us if the correlation is actually significant. 
Before we do a multiple regression we first make our text-based categories "smoker" and "sex" info numerical values, so that we may try these variables as well.

**Result**
- R-squared: 0.407 which means that our multiple regression explains 40.7% of the variance in systolic bp.
- Coefficients: Top 3 variables that have the greatest coefficients (effect) is **age**, **cholesterol** and if you have the **disease** or not.
- p-value: Looking at the p value, only **age** and **weight** have a p value less than 0.05, which means that we cannot reject our null hypothesis for the variables height, cholesterol, smoker, disease and sex. And our null hypothesis is in this case that there is no correlation between the independent variable and the dependent variable.
- Notes: In the model summary we got this note: *"The condition number is large, 4.88e+03. This might indicate that there are
strong multicollinearity or other numerical problems."* 
Multicollinearity means that some of our independent variables might be affecting the other. For example, height and weight might affect one another and disturb our results in our multiple regression.


**Source:**
- https://www.datarobot.com/blog/multiple-regression-using-statsmodels/ <-- Regression-code
- Claude for fixing a nice print format with β-symbols


In [ ]:
pd.to_numeric(df["disease"])

df['sex_numerical'] = 0
df.loc[df['sex'] == 'M', 'sex_numerical'] = 1

df['smoker_numerical'] = 0
df.loc[df['smoker'] == 'Yes', 'smoker_numerical'] = 1

use_reg = RegressionAnalysis(df)
use_reg.multiple_regression("systolic_bp", ["age", "weight", "height", "cholesterol", "smoker_numerical", "sex_numerical", "disease"])

**VIF (Variance inflation factor)**

To see how much each independent variable from our previous multiple regression is affected by the other independent variables, we do a VIF (Variance Inflation Factor) to detect multicollinarity. 

**Result**
- Smoking, sex and however the person had the disease or not, had a relatively low VIF number, which tells us that those three variables are NOT or only in a small degree subjected by collinearity of the other independent variables. But neither of these variables had a p-value < 0.05 (they were all way higher), which tells us that we cannot reject our null hypothesis for these variables anyway.
- Cholesterol, height, weight and age have an extremely high VIF number, which indicates that those variables can be predicted by the other independent variables and are therefore subjected by collinearity. 

**Source**
- https://www.geeksforgeeks.org/python/detecting-multicollinearity-with-vif-python/ 
- https://www.statology.org/how-to-calculate-vif-in-python/ <--Det mesta av koden, med liten modifikation i ordningen
- https://quantifyinghealth.com/vif-threshold/


In [ ]:
use_reg = RegressionAnalysis(df)

use_reg.calculate_vif(["age", "weight", "height", "cholesterol", "smoker_numerical", "sex_numerical", "disease"])

use_reg.calculate_vif(["age", "weight", "height", "cholesterol"])



**PCA**

We do a principal component analysis to see which variables we can cluster together to a set of new variables. We also get to see how much of the variance in our dataset is explained by our new PC-variables.

**Result**
- PC1 explains 39% of our datas variance! And the main contributors to PC1 is age and cholesterol. Therefore, we can internally name our PC1 "Age related health-factors"
- PC2 explains 32.4% of our datas variance, and the main contributors here is weight and height. Therefore, we internally name our PC2 "Physical size"
- Together PC1 and PC2 explains 71.3%, which is why we keep only these two PC-components.

So now we have two new variables PC1 (age related health-factors) and PC2 (physical size), which we can do a PCR (Principal component regression) on with our dependent variable systolic bp.

**Source**
- https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
- Claude <-- The heavy lifting of this code
- https://www.youtube.com/watch?v=rIKVoHp1HLM <--fit_transform
- https://www.youtube.com/watch?v=HWkM1kH52d8 <-- eigenvalue and eigenvector
- 

In [ ]:
pca_vars = ['age', 'height', 'weight', 'cholesterol']

use_pca_or_pcr = PCA_and_PCR(pca_vars, df)

results = use_pca_or_pcr.pca()


**PCR (Principal component regression)**

Doing a new regression with our PC variables eliminates the problem of multicollinearity. 

**Result**
- Explained variance by PC1 and PC2 of our total data is 71.3%, which means that PC1 and PC2 together explains the vast majority of the variance in our independent variables variables age, height, weight, cholesterol.
- R-squared: 32.34%, which means that about 32% of our variance in systolic bp is explained by PC1 and PC2. Altought R2 is lower in our multiple regression (about 40%) we know that PC1 and PC2 doesn't have problems with multicollinearity (VIF stable around 1). This is a really good tradeoff!
- Coefficients: PC1 ("Age related health-factors") increases systolic bloodpressure by 5.7 mmHg for every increase of 1 PC1 unit. PC2 on the other hand decreases systolic bloodpressure by 1.26 mmHg for every 1 unit increase of PC2. This tells us that age related health-factors has a positive correlation with a relatively strong effect on systolic bloodpressure (which was expected), while, interestingly, PC2 ("Physical size") has a negative correlation with systolic bloodpressure, meaning that the larger you are as a person, the lower your systolic bloodpressure will be. 

**Analysis** 
- Let's think about what constitutes PC1: age, weight and cholesterol. But what constitutes PC2? Well, there we have height and weight (again), but looking at PC2 we also see a negatively association with age. So even though it was expected that age related health-factors has a positive correlation with systolic bloodpressure, now imagine that PC2 is about a person being large (which we would generally think would lead to a higher systolic bloodpressure), but PC2 is physical size WHEN age has been removed from the equation. So maybe the people within PC2 are physically fit (which equals larger bodymass), and this leads to a higher systolic bloodpressure. Food for thought!

**Source**
- Same as PCA combined with linear regression


In [ ]:
use_pca_or_pcr.pcr("systolic_bp", 2)

**Summary of part 2**

- Linear regression: We first did 4 different linear regressions where we tried the independent variables: height, weight, cholesterol and age to see how they correlated with our dependent variable systolic bloodpressure. The results were that only age seemed to explain the variance of systolic bp. Then we did a residual diagnostics to see if our linear regression met the neccessary conditions - which it did! 
- Multiple regression: Then we did a multiple regression to see how our independent variables explained the variance in our dependent variable systolioc bp in a statistical significant way. Our results were that only age and weight had a p-value below 0.05, however, we got a note that there was probably high multicollinearity between the variables.
- VIF: To see which variables were subjected of multicollinearity, we did a VIF (variance inflation factor). The results were that only smoking and sex had a relatively low VIF, but remember from our multiple regression that smoking and sex did not give a result of statistical significance. So we redid our VIF without sex, disease and smoking, to see if we could lower our VIF for the other variables. The results were that age, weight, height and cholesterol still had way too high VIF results.
- PCA: So next, to eliminate multicollinearity, we did a PCA (principal component analysis) and came up with two new variables: PC1 ("Age related health-factors") consisting of age, weight and cholesterol, and PC2 ("Physical size") consisting of weight and height. Together PC1+PC2 explained 71.3% of our variance in our data, which makes it suitable to exchange all our independent variables for PC1 and PC2, knowing we have our multicollinearity regulated.
- PCR: And finally, we did a PCR (principal component regression) to see how our PC1 and PC2 explained the variance in systolic bp. Our results gave us that PC1 and PC2 explained 32% of the variance in systolic bloodpressure, while PC1 was positively correlated with systolic bloodpressure (with a strong effect), and PC2 was negatively correlated with systolic bloodpressure (with a mild/low effect). 
